In [3]:
# adds parent dir to python path
import sys
sys.path.insert(0, '..')

#system lib
import os

# 3rd party lib
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib auto
matplotlib.rcParams.update({'font.size': 22})

# astro lib
from astropy.io import fits
import sep

# my lib
import ImageTools as it
import DataTools as dt

Using matplotlib backend: Qt5Agg


In [15]:
def make_nonnegative(img):
    epsilon = 0 if len(img[img<0])==0 else img[img>0].min()
    return img + abs(img.min()) + epsilon

def get_random_spheroids(num=10):
    with open('../spheroids', 'r') as f:
        spheroids = np.array(f.readlines())
    
    if num==-1:
        num = len(spheroids)
    
    selected = np.random.choice(spheroids, num, replace=False)
    
    data_dir = os.path.join(os.getenv('HOME'), 'Documents/astro_data/orig_images')
    fmask = 'GDS_{}_{}.fits'
    f_string = os.path.join(data_dir, fmask)

    sources = []
    for s in selected:
        img = fits.getdata(f_string.format(s.strip(), 'h'))
        segmap = fits.getdata(f_string.format(s.strip(), 'segmap'))
        img_id = int(s.split('_')[1])
        sources.append((img.copy(), segmap.copy(), img_id))
        del img 
        del segmap
    
    return sources

# Petrosian Helpers ----------------------------------------------------------
def one_over_eta(rs, fs, R):
    return ((np.pi*R**2) / (np.sum(fs[rs<=R]))) * fs[rs==R]

def petrosian_radius(rs, fs):
    r_candidates = np.array([one_over_eta(rs, fs, R) for R in rs])
    return rs[np.square(r_candidates-0.2).argmin()]

def petrosian_flux(rs, fs, R_p):
    return np.sum(fs[rs<=2*R_p])

def R_x(rs, fs, x):
    x /= 100
    sum_ratio = np.cumsum(fs)/np.sum(fs)
    return rs[np.square(sum_ratio-x).argmin()]

def petrosian_Re(R_50, R_90):
    P_3 = 8e-6
    P_4 = 8.47
    return R_50 / (1 - P_3*(R_90/R_50)**P_4)

def get_rs_and_fs(img, src_map):
    cx, cy = it.img_center(img, src_map)
    xs, ys = np.meshgrid(np.arange(img.shape[0]), np.arange(img.shape[1]).T)
    rs = np.sqrt(np.square(xs-cx)+np.square(ys-cy))
    
    rs = rs.flatten()
    fs = img.flatten()
    sorted_rs = np.argsort(rs)
    
    rs = rs[sorted_rs]
    fs = fs[sorted_rs]
    
    #fs = dt.loessc(rs, fs, 10)
    
    return rs, fs

def get_re(img, src_map):
    rs, fs = get_rs_and_fs(img, src_map)
    
    R_p = petrosian_radius(rs, fs)
    F_p = petrosian_flux(rs, fs, R_p)
    p_mask = rs <= 2*R_p
    
    _rs = rs[p_mask]
    _fs = fs[p_mask]

    R_50 = R_x(_rs, _fs, 50)
    R_90 = R_x(_rs, _fs, 90)

    re = petrosian_Re(R_50, R_90)
    re_idx = np.square(_rs-re).argmin()
    
    return rs[re_idx], rs, fs
# Petrosian Helpers ----------------------------------------------------------

def denoise(img, segmap, img_id):
    noise_bank = img[segmap==0].flatten()
    other_source = np.logical_and(segmap!=0, segmap!=img_id)
    np.place(img, other_source, noise_bank)
    
    array_sep_likes = img.byteswap().newbyteorder()
    bkg = sep.Background(array_sep_likes, mask=src_map, bw=10, bh=10)
    img = make_nonnegative(img-bkg)
    
    return img


In [34]:
re_max_ratio = []
spheroids = get_random_spheroids(num=50)

res = []
normed_rs = []
normed_fs = []
count = 0
for img, segmap, img_id in spheroids:
    print(count/len(spheroids), end='\r')
    count += 1
    
    #background subtract images
    src_map = segmap==img_id
    
    img = denoise(img, segmap, img_id)
    
    # measure petrosian re 
    re, rs, fs = get_re(img, src_map)
    res.append(re)    
    
    # measure out to R_max=the r that sums to twice the flux in re
    re_idx = np.where(rs==re)[0][0] 
    ie = fs[re_idx]
    
    sum_to_re = fs[rs<=re_idx].sum()
    src_L = sum_to_re*2
    img_L = fs.sum()
    print(src_L, img_L, re)
    if src_L>img_L:
        re_max_ratio.append(sum_to_re/img_L)
        continue
    
    print(img_id)
    max_r = np.square(np.cumsum(fs)-src_L).argmin()
    print(max_r)
    
    max_r = rs[max_r]
    print(max_r)
    
    fs = fs[rs<=max_r]/ie
    rs = rs[rs<=max_r]/re
    
    print(rs)
    print(fs)
    
    
    normed_rs.append(rs)
    normed_fs.append(fs)
    
    # check that 5re is a valid number
#    if (5*re > rs.max()):
#        re_ratio = re/rs.max()
#        re_exceeded.append(re_ratio)
#        
#        r_mask = rs <= 5*re
#       rs = rs[r_mask]/re
#       fs = fs[r_mask]/ie
#
#        normed_rs.append((rs,re_ratio))
#        normed_fs.append(fs)
#        
#    else:
#        rs = rs/re
#        fs = fs/ie
#
#        normed_rs.append((rs,0))
#        normed_fs.append(fs)

415.447265625 207.724 7.56049941315
319.882995605 159.941 32.0893950976
1557.47802734 778.739 5.81078708541
0.0 804.982 0.312267678072
15075
0
0.312267678072
[ 1.]
[ 1.]
432.010345459 216.005 32.1059974332
231.055023193 115.528 25.2024550745
300.395263672 150.198 33.0944180439
383.335784912 191.668 33.1536114439
385.271118164 192.636 25.8242660958
371.211303711 185.606 15.8292327808
197.620635986 98.8103 13.2900726495
421.573638916 210.787 32.8861731987
506.992279053 253.496 32.7756568203
357.377563477 178.689 33.0665567251
230.75201416 115.376 32.9539650047
536.814025879 268.407 31.9279217133
335.894805908 167.947 18.3375599238
271.237701416 135.619 7.06529054372
237.910110474 118.955 8.52698308903
267.496734619 133.748 20.1947749149
323.084838867 161.542 17.1150972948
1051.04772949 525.524 12.7016778672
316.13949585 158.07 24.8014380096
472.09362793 236.047 24.8405019413
430.60836792 215.304 33.2864898644
318.209747314 159.105 10.1202943759
411.868743896 205.934 26.3999022646
637.943

In [29]:
f_size = (10,10)

plt.figure(figsize=f_size)
plt.title('Cumulative SBP')
plt.ylabel('$\sum_r^R I(r) / \sum_r^{Rmax} I(r)$')
plt.xlabel('$R/R_e$')
for r, f in zip(normed_rs, normed_fs):
    color = (e, 0, 0)
    plt.plot(r, np.cumsum(y)/y.sum(), color=color, alpha=0.25)

plt.figure(figsize=f_size)
plt.title('$R_e$ Hist')
plt.hist(res, bins=50)

plt.figure(figsize=f_size)
plt.title('$R_e$/Max Hist')
plt.hist(re_max_ratio, bins=50)


plt.show()

0
50


ValueError: x and y must have same first dimension, but have shapes (1,) and (7056,)

In [11]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.resources import CDN
from bokeh.embed import file_html
output_notebook()

Loading BokehJS ...

In [ ]:
fig = figure()

for (r,e), y in zip(normed_rs, normed_fs):
    fig.line(r.tolist(), ((np.cumsum(y)/y[r<1].sum())*0.5).tolist())
    
file_html(fig, CDN, '~/test.html')
    
show(fig, notebook_handle=True)